# Data Loading and Preprocessing

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and normalization
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Loading training, validation, and testing datasets
train_data = data_gen.flow_from_directory(
    'data/train',
    target_size=(64, 64),
    batch_size=64,
    subset='training'
)

val_data = data_gen.flow_from_directory(
    'data/val',
    target_size=(64, 64),
    batch_size=64,
    subset='validation'
)

test_data = data_gen.flow_from_directory(
    'data/test',
    target_size=(64, 64),
    batch_size=64
)

Found 634 images belonging to 2 classes.
Found 19 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


# Convolutional Neural Network

In [2]:
model = models.Sequential()

# First convolutional block
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Second convolutional block
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Third convolutional block
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional block
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flattening and dense layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

# Model summary
model.summary()

/Users/Skele/Documents/Documents - Tushar's MacBook Pro/I320D Applied ML/PROJ/Jivianne/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,726,626 (10.40 MB)

 Trainable params: 2,726,562 (10.40 MB)

 Non-trainable params: 64 (256.00 B)

# Compiling and Training

In [3]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

/Users/Skele/Documents/Documents - Tushar's MacBook Pro/I320D Applied ML/PROJ/Jivianne/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 320ms/step - accuracy: 0.5268 - loss: 1.4937 - val_accuracy: 0.5263 - val_loss: 0.6919
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step - accuracy: 0.6110 - loss: 0.6647 - val_accuracy: 0.5263 - val_loss: 0.6881
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - accuracy: 0.6499 - loss: 0.5996 - val_accuracy: 0.4211 - val_loss: 0.6998
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 263ms/step - accuracy: 0.8179 - loss: 0.4512 - val_accuracy: 0.4737 - val_loss: 0.7099
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.8230 - loss: 0.3578 - val_accuracy: 0.2632 - val_loss: 0.7131
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.8760 - loss: 0.2879 - val_accuracy: 0.4737 - val_loss: 0.7199
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 267ms/step - accuracy: 0.9621 - loss: 0.1251 - val_accuracy: 0.4737 - val_loss: 0.7784
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 258ms/step - accuracy: 0.9770 - loss: 0.0769 - val_accuracy: 0.

In [4]:
# Evaluating model accuracy on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.4f}")

model.save("modelCNN.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.8160 - loss: 0.5360
Test Accuracy: 0.8100
